# Welcome to Thermodynamics - Too Hot  to Handle 

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy as sp
from thermo import *

kb = 1.380649e-23  # J/K
Mkb = 1.380649e-29 # MJ/K
Na = 6.02214076e23  # mol^-1
R = kb * Na  # J/(mol*K)
MR = Mkb * Na  # MJ/(mol*K)
g = 9.81  # m/s^2
hplanck = 6.62607015e-34  # J*s or J*Hz^-1
Mhplanck = hplanck * 1e-6  # MJ*s or MJ*Hz^-1
c = 299792458  # m/s

In [ ]:
# Example usage of the apdx_function module:
print("Retrieving APDX 1 data for CO2:")
print(get_apdx_1('CO2', 'R', True))

# Apdx 4 example: get gamma for Air at 300K
print("\nRetrieving APDX 4 data for Air:")
print(get_apdx_4('Air', 'T', 300, 'gamma'))

print("\nRetrieving APDX 7 data for Air:")
print(get_apdx_7('T', 300, 'Pr'))

print("\nRetrieving APDX 8 data for saturated Water:")
print(get_apdx_8ab('Pressure', 'P', 0.50, 'vf'))

print("\nRetrieving APDX 9 data for saturated R134a:")
print(get_apdx_9ab('Pressure', 'P', 0.50, 'vf'))

In [ ]:
x = x_from_PT_and_var(8, 'P', 0.3, 's', 5)
print(x)
hf, hg = get_apdx_8ab('Pressure', 'P', 0.3, 'hf'), get_apdx_8ab('Pressure', 'P', 0.3, 'hg')
heat = (1-x) * (hg - hf)
print(heat, 'kJ/kg')
print('')
# Vars from quality and pressure far saturated water
vars_from_x_and_PT(8, 'P', 0.3, 0.6255897445536738)
print('')
vars = vars_from_x_and_quality_var(8, 0.6255897445536738, 's', 5)

In [ ]:
# Question 9

h2 = get_apdx_8ab('Pressure', 'P', 0.2, 'hf')
h1 = h2 + 5000/2
print('h1:', h1)
print('T1:', get_apdx_8c(('P', 'h'), (10, h1), 'T'))

In [ ]:
# Question 10

print(get_apdx_7('T', 300, 'Pr'))
print(get_apdx_7('Pr', 30.492, 'h'))
print(get_apdx_7('T', 300, 'h'))
print(get_apdx_7('h', 799.71, 'T'))

In [ ]:
# Question 11

gamma = 1.044
M = 114.23 # g/mol
M = M * 1e-3 # kg/mol
Rs = R/M # J/(kg*K)

P1 = 1e6 # Pa
P2 = 2e6 # Pa
v1 = 0.6 # m**3/kg

# Finding v2 through isentropic relations
v2 = v1 * (P1/P2)**(1/gamma)

# The work of the isentropic process is giben by
# W = (P2*v2 - P1*v1)/(gamma-1)
W = (P2*v2 - P1*v1)/(gamma-1)
print(f'W = {W:.2f} J/kg')

# alternative 2 step process to the isentropic one from 1 to 2
# Ideal gas law
T1 = P1 * v1 / Rs

# step 1 to 1b is isothermal
W1 = -Rs * T1 * np.log(v2/v1)
print(f'W1 = {W1:.2f} J/kg')

# step 1b to 2 is isochoric, so W2 = 0
W2 = 0
Wb = W1 + W2
print(f'Wb = {Wb:.2f} J/kg')

# diff between W and Wb
Wdiff = W - Wb
print(f'Wdiff = {Wdiff:.2f} J/kg')

In [ ]:
print(h_air(1976.4))
print(h_air(988.2))

In [ ]:
from thermo.diesel_solver import solve_diesel_cycle, define_empty_variables

# Example usage of the diesel_solver module
variables = define_empty_variables()

variables[0]['r'] = 20  # Compression ratio
variables[0]['rc'] = 2.0  # Cut-off ratio
variables[1]['T1'] = 298.15  # K
variables[1]['P1'] = 100  # kPa

solve_diesel_cycle(variables)

In [ ]:
from thermo.otto_solver import define_empty_variables, solve_otto_cycle, otto_display_tables

vars = define_empty_variables()

# give the solver *one* piece of information per state
vars['1']['P'] = 100 # kPa
vars['1']['T'] = 295 # K
vars['3']['T'] = 2000 # K
#vars['Qh'] = 900
vars['r'] = 9 # compression ratio

solved = solve_otto_cycle(vars, cold_air_standard=True)
otto_display_tables(solved, sig_figs=5)

In [ ]:
from thermo.Rankine_solver import define_empty_variables, solve_r_rankine_cycle
from thermo.Rankine_solver import plot_Ts_cycle, rankine_display_tables

vars = define_empty_variables()

# give the solver *one* piece of information per state
vars['2']['T'] = -8  # C
vars['3']['P'] = 0.8 # MPa
vars['m_dot'] = 0.1 # kg/s

solved = solve_r_rankine_cycle(vars)

rankine_display_tables(solved)
plot_Ts_cycle(solved)